In [1]:
import Io_tf_binary_general as io

Welcome to JupyROOT 6.15/01


In [2]:
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
#najsamwpierw tworze swoje wczytywacze

In [25]:
wczytywacz_tlo=io.Io_tf_binary_general("tlo",'r')

{'leg_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'BJetBetaScore': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsMassTrans': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsPT': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'nJets30': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'visMass': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_charge': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_combreliso': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': FixedLenFeature(shape=[1], dtype=tf.float32, de

In [26]:
wczytywacz_syg=io.Io_tf_binary_general("sygnal",'r')

{'leg_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'BJetBetaScore': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsMassTrans': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsPT': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'nJets30': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'visMass': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_charge': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_combreliso': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': FixedLenFeature(shape=[1], dtype=tf.float32, de

In [7]:
wczytywacz_syg.types()#tutaj widac, co tam jest, chyba
#nie widac ze jest label, bo wiadomo ze jest label 
# ( to znaczy odrozninie tla od sygnalu)

{'leg_0_momentum': [4, 'f'],
 'leg_1_momentum': [4, 'f'],
 'jet_0_momentum': [4, 'f'],
 'jet_1_momentum': [4, 'f'],
 'BJetBetaScore': [1, 'f'],
 'higgsMassTrans': [1, 'f'],
 'higgsPT': [1, 'f'],
 'nJets30': [1, 'f'],
 'visMass': [1, 'f'],
 'leg_1_charge': [1, 'f'],
 'leg_1_combreliso': [1, 'f'],
 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': [1, 'f'],
 'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': [1, 'f'],
 'leg_2_charge': [1, 'f'],
 'leg_2_decayMode': [1, 'f']}

In [18]:
syg_dataset=wczytywacz_syg.read()

In [19]:
tlo_dataset=wczytywacz_tlo.read()

In [27]:
#mozna by to napisac lepiej, to znaczy mozna by robic 
#zbachowane rzeczy i te zbachowane po jednym dawac.
def generator_z_dataset(dataset,czy_shuffle,ile_w_buferze,czy_repeat):
    #takie tam tensorflowowe rzeczy
    #ile_w_buferze ma znaczenie tylko jak czy_shuffle==true
    if(czy_shuffle):
        dataset=dataset.shuffle(ile_w_buferze)
    if(czy_repeat):
        dataset=dataset.repeat() #bedzie krazyc w kolko
    iterator=dataset.make_one_shot_iterator()
    f,l=iterator.get_next()
    with tf.Session() as sess:
        while True:
            yield(sess.run({"features":f,"l":l}))
def do_listy(dataset):
    gen=generator_z_dataset(dataset,False,0,False)
    wyrzut=[]
    while True:
        try:
            wyrzut.append(next(gen))
        except:
            print("koniec danych")
            break
    return wyrzut

In [15]:
#gen_syg=generator_z_dataset(syg_dataset,False,0,False)

In [16]:
#gen_tlo=generator_z_dataset(tlo_dataset,True,1000,True)

In [28]:
lista_tlo=do_listy(wczytywacz_tlo.read())

koniec danych


In [29]:
lista_syg=do_listy(wczytywacz_syg.read())

koniec danych


In [30]:
io_train=io.Io_tf_binary_general("train",'w')
io_val=io.Io_tf_binary_general("val",'w')
io_test=io.Io_tf_binary_general("test",'w')

In [33]:
def wrzuc_do_pliku_mieszajac(io_obiekt,lista_syg,lista_tlo,od_kad,do_kad):
    lista_syg=lista_syg[int(od_kad*len(lista_syg)):int(do_kad*len(lista_syg))]
    lista_tlo=lista_tlo[int(od_kad*len(lista_tlo)):int(do_kad*len(lista_tlo))]
    for i in range(len(lista_syg)):
        io_obiekt.write_general(**lista_syg[i])
        io_obiekt.write_general(**lista_tlo[i % len(lista_tlo)])
    io_obiekt.close()

In [34]:
wrzuc_do_pliku_mieszajac(io_train,lista_syg,lista_tlo,0.,0.8)
wrzuc_do_pliku_mieszajac(io_val,lista_syg,lista_tlo,0.8,0.9)
wrzuc_do_pliku_mieszajac(io_test,lista_syg,lista_tlo,0.9,1.)


In [37]:
#konczy sie out_of_range error, tak ma byc
"""
for przypadek_syg in gen_syg:
    przypadek_tlo=next(gen_tlo)
    wymieszane.write_general(**przypadek_syg)
    wymieszane.write_general(**przypadek_tlo)
    """
    

'\nfor przypadek_syg in gen_syg:\n    przypadek_tlo=next(gen_tlo)\n    wymieszane.write_general(**przypadek_syg)\n    wymieszane.write_general(**przypadek_tlo)\n    '

In [38]:
czytanie_sprawdzanie=io.Io_tf_binary_general("train",'r')

{'BJetBetaScore': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsMassTrans': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'higgsPT': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'jet_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'jet_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_0_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_1_charge': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_combreliso': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_1_momentum': FixedLenFeature(shape=[4], dtype=tf.float32, default_value=None), 'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None), 'leg_2_charge': FixedLenFe

In [39]:
spr=generator_z_dataset(czytanie_sprawdzanie.read(),False,0,False)

In [42]:
#wielokrotnie klikajac widac,ze dziala
next(spr)

{'features': {'BJetBetaScore': array([-999.], dtype=float32),
  'higgsMassTrans': array([4.9182773], dtype=float32),
  'higgsPT': array([50.86785], dtype=float32),
  'jet_0_momentum': array([ 318.45612 ,   -8.966505,   18.99214 , -317.72153 ], dtype=float32),
  'jet_1_momentum': array([0., 0., 0., 0.], dtype=float32),
  'leg_0_momentum': array([ 88.02361 ,  30.906635, -21.522163, -79.55955 ], dtype=float32),
  'leg_1_charge': array([1.], dtype=float32),
  'leg_1_combreliso': array([0.01387171], dtype=float32),
  'leg_1_momentum': array([ 72.39734 , -40.715553,  32.471966, -50.164165], dtype=float32),
  'leg_2_byCombinedIsolationDeltaBetaCorrRaw3Hits': array([0.], dtype=float32),
  'leg_2_byIsolationMVArun2v1DBoldDMwLTraw': array([0.9711645], dtype=float32),
  'leg_2_charge': array([-1.], dtype=float32),
  'leg_2_decayMode': array([1.], dtype=float32),
  'nJets30': array([0.], dtype=float32),
  'visMass': array([93.22299], dtype=float32)},
 'l': 1}